In [2]:
import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Conv2DTranspose
import imageio
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import scipy.io as sio

# Turn inFile into a one hot label
def createOneHot(inFile,amountOfClasses):
    outFile = np.zeros((inFile.shape[0],inFile.shape[1], amountOfClasses));
    for i in range(0,inFile.shape[0]):
        for j in range(0,inFile.shape[1]):
            #if inFile[i,j] != 0:
            outFile[i,j,int(inFile[i,j]-1)] = 1;
    return outFile;

# Turn 41 orthomosaics into one array. From 2 separate paths, 1-25 and 1-16
def loadOneOrthomosaic(imagePath1,imagePath2):
    temp= imageio.imread(imagePath1 + '1.png');
    inputData = np.zeros( (temp.shape[0],temp.shape[1],inputChannels) ,dtype=np.dtype('uint8'))
    for i in range(1,25):
        temp= imageio.imread(imagePath1 + str(i) + '.png');
        inputData[:,:,i] = temp[:,:,0];
    for i in range(1,16):
        temp= imageio.imread(imagePath2 + str(i) + '.png');
        inputData[:,:,i+25] = temp[:,:,0];
    return np.expand_dims(inputData,axis=0)

# Turn 41 orthomosaics into one array. From 1 path with Bands 1 to 41
def loadOneOrthomosaic(imagePath):
    temp= imageio.imread(imagePath + '1.png');
    inputData = np.zeros(( temp.shape[0], temp.shape[1], inputChannels),dtype=np.dtype('uint8'))
    for i in range(1,41):
        temp= imageio.imread(imagePath + str(i) + '.png');
        inputData[:,:,i] = temp[:,:];
    return np.expand_dims( inputData, axis=0)

# Load one label from a .mat
def loadOneLabel(imagePath):
    temp = sio.loadmat(imagePath);
    trainOneHotLabels =  createOneHot( temp['labeledPicture'], amountOfClasses);
    return np.expand_dims( trainOneHotLabels, axis=0)

# Read training data and labels for an array of path tuples, returns (data,labels)
def readSet(paths):
    numberOfImages = len(paths);
    data = np.zeros( (numberOfImages,imageSize[1],imageSize[0],inputChannels), dtype=np.dtype('uint8'));
    labels = np.zeros((numberOfImages,imageSize[1],imageSize[0],amountOfClasses), dtype=np.dtype('uint8'))
    for i in range(0,numberOfImages):
        data[i,:,:,:] = loadOneOrthomosaic(paths[i][0]);
        labels[i,:,:,:] = loadOneLabel(paths[i][1]);
    return data,labels

def oneHotToCategorical(oneHotLabel):
    temp = np.zeros((oneHotLabel.shape[1],oneHotLabel.shape[2]), dtype=np.dtype('uint8') );
    for i in range(0,oneHotLabel.shape[1]):
        for j in range(0,oneHotLabel.shape[2]):
            #temp[i,j] = np.argmax(oneHotLabel[0,i,j,:]) + 1;
            temp[i,j] = np.argmax(oneHotLabel[0,i,j,:]);
    return temp

def predictAndSaveOneImage(inPath,outPath,FCNmodel):
    testImage = loadOneOrthomosaic(inPath);
    testLabelPredict = FCNmodel.predict(testImage);
    testLabelPredictCat = oneHotToCategorical(testLabelPredict);
    sio.savemat(outPath, mdict={'testLabelPredict': testLabelPredictCat});

def predictAndSaveSet(pathArray, FCNmodel):
    numberOfImages = len(pathArray);
    for i in range(0,numberOfImages):
        predictAndSaveOneImage(pathArray[i][0], pathArray[i][1], FCNmodel)
        print('Printed ' + pathArray[i][1])
        
def plotImage(image):
    plt.imshow(image)

Using TensorFlow backend.


In [3]:
basePath = '/Volumes/mac_jannic_2017/thanujan/Datasets/xFcnClassifier/';

imagePath1 = basePath + 'Data/FIP/20170622/Band';
labelpath1 = basePath + 'Labels/FIP20170622.mat';
imagePath2 = basePath + 'Data/Ximea_Tamron/20170510/Band';
labelpath2 = basePath + 'Labels/XimeaT20170510.mat';
imagePath3 = basePath + 'Data/Ximea_Tamron/20170622/Band';
labelpath3 = basePath + 'Labels/XimeaT20170622.mat';

trainPaths = [[imagePath1,labelpath1],[imagePath2,labelpath2],[imagePath3,labelpath3]];
del imagePath1,labelpath1,imagePath2,labelpath2,imagePath3,labelpath3,basePath

amountOfClasses = 9;
inputChannels = 41;
imageSize = [1600,1600]; # [X,Y]

In [3]:
model = Sequential([
    Conv2D(20, 5, input_shape=(imageSize[0], imageSize[1], inputChannels), activation='relu'),
    MaxPooling2D(),
    Conv2DTranspose(20, 6, strides=2, activation='relu'),
    Conv2D(amountOfClasses, (1,1), activation='softmax')
])
model.summary()
model.compile(optimizer='adagrad', loss='categorical_crossentropy', metrics=['accuracy']);

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 1596, 1596, 20)    20520     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 798, 798, 20)      0         
_________________________________________________________________
conv2d_transpose_1 (Conv2DTr (None, 1600, 1600, 20)    14420     
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 1600, 1600, 8)     168       
Total params: 35,108
Trainable params: 35,108
Non-trainable params: 0
_________________________________________________________________


In [4]:
model2Conv = Sequential([
    Conv2D(128, 3, input_shape=(imageSize[0], imageSize[1], inputChannels), activation='relu'),
    Conv2D(128, 3, activation='relu'),
    MaxPooling2D(),
    Conv2DTranspose(20, 6, strides=2, activation='relu'),
    Conv2D(amountOfClasses, (1,1), activation='softmax')
])
model2Conv.summary()
model2Conv.compile(optimizer='adagrad',
              loss='categorical_crossentropy',
              metrics=['accuracy']);

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 1598, 1598, 128)   47360     
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 1596, 1596, 128)   147584    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 798, 798, 128)     0         
_________________________________________________________________
conv2d_transpose_1 (Conv2DTr (None, 1600, 1600, 20)    92180     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 1600, 1600, 9)     189       
Total params: 287,313
Trainable params: 287,313
Non-trainable params: 0
_________________________________________________________________


In [5]:
# Load the training data and labels
trainData,trainLabels = readSet(trainPaths)

In [6]:
whos

Variable                 Type          Data/Info
------------------------------------------------
Conv2D                   type          <class 'keras.layers.convolutional.Conv2D'>
Conv2DTranspose          type          <class 'keras.layers.conv<...>utional.Conv2DTranspose'>
Image                    module        <module 'PIL.Image' from <...>e-packages/PIL/Image.py'>
MaxPooling2D             type          <class 'keras.layers.pooling.MaxPooling2D'>
Sequential               type          <class 'keras.models.Sequential'>
amountOfClasses          int           9
createOneHot             function      <function createOneHot at 0x107caf7b8>
imageSize                list          n=2
imageio                  module        <module 'imageio' from '/<...>ges/imageio/__init__.py'>
inputChannels            int           41
keras                    module        <module 'keras' from '/Us<...>kages/keras/__init__.py'>
loadOneLabel             function      <function loadOneLabel at 0x11a3da840>
l

In [7]:
from keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='acc', patience=1);

#model2Conv.fit(trainData, trainLabels, epochs=3,callbacks=[early_stopping]);
model2Conv.fit(trainData, trainLabels, epochs=3);

Epoch 1/3
3/3 [==============================] - 777s 259s/step - loss: 4.9293 - acc: 0.0155
Epoch 2/3
3/3 [==============================] - 3128s 1043s/step - loss: 4.7401 - acc: 0.7958
Epoch 3/3
3/3 [==============================] - 2605s 868s/step - loss: 4.2855 - acc: 0.6143


In [8]:
#model.save('networkModels/FCN-None-None-conv5.h5')
model2Conv.save('networkModels/FCN-1600-1600-128conv3-128conv3.h5')

In [9]:
basePath = '/Volumes/mac_jannic_2017/thanujan/Datasets/xFcnClassifier/';

inPath1 = basePath + 'testData/Ximea_Tamron/20170613/Band';
outpath1 = basePath + 'testLabelPredict/XimeaT_20170613.mat';
inPath2 = basePath + 'testData/FIP/20170531/Band';
outpath2 = basePath + 'testLabelPredict/FIP_20170531.mat';
inPath3 = basePath + 'testData/FIP/20170622/Band';
outpath3 = basePath + 'testLabelPredict/FIP_20170622.mat';
inPath4 = basePath + 'testData/FIP/20170802/Band';
outpath4 = basePath + 'testLabelPredict/FIP_20170802.mat';

testPaths = [[inPath1,outpath1],[inPath2,outpath2],[inPath3,outpath3],[inPath4,outpath4]];

#predictAndPlotSet(testPaths, model2Conv);
predictAndSaveSet(testPaths, model2Conv);

Printed /Volumes/mac_jannic_2017/thanujan/Datasets/xFcnClassifier/testLabelPredict/XimeaT_20170613.mat
Printed /Volumes/mac_jannic_2017/thanujan/Datasets/xFcnClassifier/testLabelPredict/FIP_20170531.mat
Printed /Volumes/mac_jannic_2017/thanujan/Datasets/xFcnClassifier/testLabelPredict/FIP_20170622.mat
Printed /Volumes/mac_jannic_2017/thanujan/Datasets/xFcnClassifier/testLabelPredict/FIP_20170802.mat


In [62]:
import pydot_ng as pydot
import graphviz
from keras.utils import plot_model
plot_model(model2Conv, to_file='model2Conv.png')

ImportError: Failed to import pydot. You must install pydot and graphviz for `pydotprint` to work.

In [59]:
import pydot_ng as pydot
import graphviz
print(pydot.find_graphviz())

{'dot': '/Users/jveith/anaconda/bin/dot', 'twopi': '/Users/jveith/anaconda/bin/twopi', 'neato': '/Users/jveith/anaconda/bin/neato', 'circo': '/Users/jveith/anaconda/bin/circo', 'fdp': '/Users/jveith/anaconda/bin/fdp', 'sfdp': '/Users/jveith/anaconda/bin/sfdp'}


In [70]:
whos

Variable                 Type             Data/Info
---------------------------------------------------
Conv2D                   type             <class 'keras.layers.convolutional.Conv2D'>
Conv2DTranspose          type             <class 'keras.layers.conv<...>utional.Conv2DTranspose'>
EarlyStopping            type             <class 'keras.callbacks.EarlyStopping'>
Image                    module           <module 'PIL.Image' from <...>e-packages/PIL/Image.py'>
MaxPooling2D             type             <class 'keras.layers.pooling.MaxPooling2D'>
Sequential               type             <class 'keras.models.Sequential'>
a                        AxesImage        AxesImage(112.68,36;217.44x217.44)
amountOfClasses          int              8
basePath                 str              /Volumes/mac_jannic_2017/<...>/Datasets/xFcnClassifier/
createOneHot             function         <function createOneHot at 0x13773e840>
dot                      Digraph          // The Round Table\ndigra<..

In [72]:
temp = sio.loadmat(trainPaths[0][1]);
print(trainPaths[0][1])

/Volumes/mac_jannic_2017/thanujan/Datasets/xFcnClassifier/testLabelPredict/XimeaT_20170613.mat


In [71]:
temp

{'__globals__': [],
 '__header__': b'MATLAB 5.0 MAT-file Platform: posix, Created on: Fri Feb 16 19:34:09 2018',
 '__version__': '1.0',
 'testLabelPredict': array([[5, 5, 5, ..., 5, 5, 5],
        [5, 5, 5, ..., 5, 5, 5],
        [5, 5, 5, ..., 5, 5, 5],
        ...,
        [5, 5, 5, ..., 5, 5, 5],
        [5, 5, 5, ..., 5, 5, 5],
        [5, 5, 5, ..., 5, 5, 5]], dtype=uint8)}